$19^{th}$ Aug 2021

In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from datetime import datetime



In [2]:
#load the excel sheet containing the variants information

df_meta_seq = pd.read_excel('/home/douso/SarsGenomics/Processed/metadata-seq-MS-Merge@KB8.xlsx', usecols=['S_NUM', 'S', 'DT_CONF', 'G_COV'])

In [3]:
df_meta_seq.head()

,S_NUM,DT_CONF,G_COV,S
0,COVC00854,2020-06-13 00:00:00,94.8,"D614G,"
1,COVC00867,2020-06-13 00:00:00,66.7,"D614G, M900fs, M900fs"
2,COVC00893,2020-06-13 00:00:00,82.6,D614G
3,COVC00915,2020-06-13 00:00:00,NaN,NaN
4,COVC00962,2020-06-14 00:00:00,93.3,D614G


In [117]:
df_meta_seq80 = df_meta_seq.replace(pd.NA, np.nan)[df_meta_seq['G_COV'] >= 80]#.sort_values('DT_CONF')

In [129]:
df_data = df_meta_seq80[df_meta_seq80.index.isin(df_meta_seq80['DT_CONF'].dropna().index)]

In [146]:
df_data_dt = df_data.assign(DT_CONF=df_data['DT_CONF'].apply(lambda x: pd.to_datetime(x, dayfirst=True))).sort_values('DT_CONF')

In [136]:
df_data_dt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 713 entries, 0 to 1069
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   S_NUM    713 non-null    object        
 1   DT_CONF  713 non-null    datetime64[ns]
 2   G_COV    713 non-null    float64       
 3   S        710 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 27.9+ KB


In [147]:
df_data_dt.tail()

,S_NUM,DT_CONF,G_COV,S
1066,COVC24214,2021-08-23,96.7,"T19R, L452R, T478K, D614G, P681R, D950N,"
1067,COVC24215,2021-08-23,90.1,"L452R, T478K, D614G, P681R, D950N, S974S,"
1069,COVC24217,2021-08-23,93.4,"T19R, L452R, T478K, Q613H, D614G, P681R, D950N..."
936,KEMRI_02_91932,2021-10-02,98.8,"G257S, D614G, P681H"
896,KEMRI_01_89581,2021-12-01,96.5,"H69_V70del, D138H, Y145del, N501Y, A570D, D614..."


	S_NUM	DT_CONF	G_COV	S
1066	COVC24214	2021-08-23	96.7	T19R, L452R, T478K, D614G, P681R, D950N,
1067	COVC24215	2021-08-23	90.1	L452R, T478K, D614G, P681R, D950N, S974S,
1069	COVC24217	2021-08-23	93.4	T19R, L452R, T478K, Q613H, D614G, P681R, D950N...
936	KEMRI_02_91932	2021-10-02	98.8	G257S, D614G, P681H
896	KEMRI_01_89581	2021-12-01	96.5	H69_V70del, D138H, Y145del, N501Y, A570D, D614...

In [148]:
df_fort = df_data_dt.resample(rule='2W', closed='right', on='DT_CONF')

In [267]:
df_spike_trend = pd.DataFrame()
count = 1
observed_mut = []
for i,df in df_fort:
#     print(i, df)
    spike_mut_list = []
    for row in df.itertuples():
        if isinstance(row.S, str):
            mut_list = row.S.rstrip(', ').split(', ')
        else:
            pass#mut_list = []; mut_list.append(pd.NA)
        spike_mut_list.extend(mut_list)
    mut_count = Counter(spike_mut_list)
    df_new = pd.DataFrame(mut_count, index=[i])
    new_col = set(df_new.columns)
    output = sorted(list(new_col)); observed_mut.append(output)
    prev_col = set(df_spike_trend.columns)
    new_m_prev_col = new_col - prev_col
    new_u_prev_col = new_col & prev_col
    prev_m_new_prev_col = prev_col - new_u_prev_col
    if new_m_prev_col == new_col:
        if count == 1:
            output = sorted(list(new_col))#; observed_mut.append(output)
            print(f'Initially detected mutation(s) as of {i.strftime("%Y-%m-%d")} are:\n  {output}\nThe counts:\n  {dict([(k, mut_count[k]) for k in output])}')
        else:
            output = sorted(list(new_col))#; observed_mut.append(output)
            print(f'No new mutation(s) were detected as of {i.strftime("%Y-%m-%d")}')
    elif len(new_m_prev_col) > 0:
        output = sorted(list(new_m_prev_col))#; observed_mut.append(output)
        print(f'The following new mutation(s) identified as of {i.strftime("%Y-%m-%d")}:\n  {output}\nThe counts:\n  {dict([(k, mut_count[k]) for k in output])}')
    elif len(new_m_prev_col) == 0:
        output = sorted(list(prev_col))#; observed_mut.append(output)
        print(f'No new mutation(s) were detected as of {i.strftime("%Y-%m-%d")}')

    count += 1
    df_spike_trend = df_spike_trend.append(df_new)


Initially detected mutation(s) as of 2020-06-14 are:
  ['D614G']
The counts:
  {'D614G': 5}
The following new mutation(s) identified as of 2020-06-28:
  ['G1251V', 'N679K', 'S637S', 'T1116N']
The counts:
  {'G1251V': 1, 'N679K': 1, 'S637S': 1, 'T1116N': 1}
The following new mutation(s) identified as of 2020-07-12:
  ['A263S']
The counts:
  {'A263S': 1}
The following new mutation(s) identified as of 2020-07-26:
  ['F515F', 'L18F', 'N928N', 'R21I', 'S1252F', 'T618T']
The counts:
  {'F515F': 3, 'L18F': 1, 'N928N': 1, 'R21I': 1, 'S1252F': 4, 'T618T': 1}
The following new mutation(s) identified as of 2020-08-09:
  ['A845S']
The counts:
  {'A845S': 1}
The following new mutation(s) identified as of 2020-08-23:
  ['A846S', 'N394I', 'P728P', 'S691S', 'Y28Y']
The counts:
  {'A846S': 1, 'N394I': 1, 'P728P': 4, 'S691S': 1, 'Y28Y': 1}
No new mutation(s) were detected as of 2020-09-06
No new mutation(s) were detected as of 2020-09-20
The following new mutation(s) identified as of 2020-10-04:
  ['D79

### **Regions of interest mutation on spike**
319-541 - RBD
613-705 - S1/S2 junction

In [450]:
window_size = 3
interest_muts = []
for i in range(len(observed_mut) - window_size):
    period_slice = []; period_slice.extend(observed_mut[i:i+window_size])
#     print(len(period_slice))
    counter = Counter()
    for period in period_slice: 
#         print(len(period))
        cnt = Counter(period)
        counter += cnt
#         print(counter)
    for k, v in counter.items(): 
#         print(k,v)
        if v == len(period_slice):
#             if k not in interest_muts:
            interest_muts.append(k)
#             else: pass
        else: pass
print(f'The following mutations consistently appeared across {window_size} periods (are they emerging variants?):\n{dict(Counter(interest_muts))}')
#     print(period_slice)

moc_dict = dict(Counter(interest_muts))
moc_set = set(list(range(319,542)) + list(range(613,706)))
moc_dict_keys = set(moc_dict.keys())

moc_p = re.compile(r'[3-7][0-9]{2,2}')
moc_mut = set(map(lambda x: int(x), moc_p.findall(str(moc_dict_keys))))

moc = moc_set & moc_mut
if len(moc) > 0:
    p = re.compile('|'.join([str(i) for i in moc]))
    moc_fin = {k: v for k,v in moc_dict.items() if p.findall(k)}
    print(f'\n\nThe following consistent mutations occur in either the RBD (319-541) or the S1-S2 (613-705) spike regions:\n {moc_fin}')

The following mutations consistently appeared across 3 periods (are they emerging variants?):
{'D614G': 23, 'G1085E': 2, 'P728P': 3, 'V1228L': 2, 'D571D': 2, 'R683R': 2, 'D80Y': 2, 'I1114I': 1, 'L1200L': 1, 'L54F': 1, 'L841L': 1, 'P681H': 8, 'Q1201K': 1, 'S375S': 2, 'T73I': 1, 'Y145del': 8, 'T716I': 8, 'A570D': 7, 'D1118H': 7, 'H69_V70del': 7, 'E484K': 3, 'N501Y': 6, 'S982A': 6, 'A67V': 2, 'F1062F': 2, 'F888L': 3, 'Q52R': 1, 'Q677H': 2, 'A1190A': 2, 'A845A': 1, 'E583D': 1, 'F157L': 1, 'H146H': 1, 'P681R': 6, 'Q613H': 2, 'V367F': 1, 'D138H': 2, 'G1251V': 2, 'S1252C': 2, 'D950N': 3, 'E156_R158delinsG': 2, 'L452R': 3, 'T19R': 3, 'T478K': 3}


The following consistent mutations occur in either the RBD (319-541) or the S1-S2 (613-705) spike regions:
 {'D614G': 23, 'R683R': 2, 'P681H': 8, 'S375S': 2, 'E484K': 3, 'N501Y': 6, 'Q677H': 2, 'P681R': 6, 'Q613H': 2, 'V367F': 1, 'L452R': 3, 'T478K': 3}


In [266]:
df_spike_trend.sum().sort_values(ascending=False)[:10]

D614G         681.0
Y145del       209.0
H69_V70del    207.0
P681H         203.0
D1118H        194.0
A570D         190.0
S982A         188.0
N501Y         173.0
T716I         165.0
P681R         108.0
dtype: float64